In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
data=[]

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.imdb.com/interest/in0000034/")

elems = driver.find_elements(By.XPATH,"//a[contains(@class,'ipc-poster-card__title ipc-poster-card__title--clamp-2 ipc-poster-card__title--clickable')]")
hrefs = [elem.get_attribute("href") for elem in elems]

for href in hrefs:
    review_url = href.split("?")[0] + "reviews"
    driver.get(review_url)
    time.sleep(2)

    for _ in range(5):
        try:
            # Wait for the button to be clickable
            load_more = wait.until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(),'25 more')]]"))
            )
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            try:
                driver.execute_script("arguments[0].click();", load_more)
            except Exception as e:
                print("Fallback click also failed:", e)

            time.sleep(2)
        except Exception as e:
            print("NOTAVAIL", e)
            break

    reviews = driver.find_elements(By.CLASS_NAME, "ipc-html-content-inner-div")
    ratings = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")
    for i, (rating, review) in enumerate(list(zip(ratings, reviews))[:115]):
        data.append({
            "Rating": rating.text,
            "Review": review.text
        })
df = pd.DataFrame(data)
df.to_csv('datasets/initial_dataset.csv', index=False)
driver.quit()

In [ ]:
data=[]

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.imdb.com/chart/bottom/")

elems = driver.find_elements(By.XPATH,"//a[contains(@class,'ipc-title-link-wrapper')]")
hrefs = [elem.get_attribute("href") for elem in elems][:100]

for href in hrefs:
    review_url = href.split("?")[0] + "reviews"
    driver.get(review_url)
    time.sleep(2)

    for _ in range(5):
        try:
            # Wait for the button to be clickable
            load_more = wait.until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(),'25 more')]]"))
            )
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            try:
                driver.execute_script("arguments[0].click();", load_more)
            except Exception as e:
                print("Fallback click also failed:", e)

            time.sleep(2)
        except Exception as e:
            print("NOTAVAIL", e)
            break

    reviews = driver.find_elements(By.CLASS_NAME, "ipc-html-content-inner-div")
    ratings = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")
    for i, (rating, review) in enumerate(list(zip(ratings, reviews))[:100]):
        data.append({
            "Rating": rating.text,
            "Review": review.text
        })
df = pd.DataFrame(data)
df.to_csv('datasets/negative_skew.csv', index=False)

driver.quit()

In [ ]:
df1 = pd.read_csv('datasets/initial_dataset.csv')
df2 = pd.read_csv('datasets/negative_skew.csv')

final_df = pd.concat([df1, df2], ignore_index=True)
final_df.to_csv('datasets/combined_dataset.csv', index=False)